In [1]:
import numpy 
import sys 
import nmslib 
import time 
import math 
from scipy.sparse import csr_matrix 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
print(sys.version)
print("NMSLIB version:", nmslib.__version__)


In [2]:
# Just a function to read sparse data
def read_data(filename, max_qty = None): 
    row = [] 
    col = [] 
    data = [] 
    read_qty = 0 
    with open(filename,'r') as f:  
        read_num_ft = 0
        for line in f: 
            x = line.replace(':', ' ').strip().split() 
            if (len(x) % 2 != 0):
                raise(Exception('Poorly formated line %d in file %s' % (read_qty + 1, filename)))
            if (len(x) == 0): continue
            for i in range(0, len(x), 2):     
                row.append(read_qty) 
                feat_id = int(x[i])
                read_num_ft = max(read_num_ft, feat_id + 1)
                col.append(feat_id) 
                data.append(float(x[i+1])) 

            read_qty = read_qty+1 
            if max_qty != None and read_qty >= max_qty: break
            
    print('Read %d rows, # of features %d' %  (read_qty, read_num_ft))
    ft_mat = csr_matrix((numpy.array(data), (numpy.array(row), numpy.array(col))), 
                         shape=(read_qty, read_num_ft)) 
    return (read_qty, ft_mat)

In [3]:
# Read data points
(all_qty, all_data_matrix) = read_data('../../sample_data/sparse_wiki_5K.txt') 

Read 5000 rows, # of features 100000


In [4]:
# Create a held-out query data set
(data_matrix, query_matrix) = train_test_split(all_data_matrix, test_size = 0.1)

In [5]:
print("# of queries %d, # of data points %d"  % (query_matrix.shape[0], data_matrix.shape[0]) )

# of queries 500, # of data points 4500


In [6]:
# Set index parameters
# These are the most important onese
M = 30
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}

In [7]:
# Number of neighbors 
K=100

In [8]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space='cosinesimil_sparse', data_type=nmslib.DataType.SPARSE_VECTOR) 
index.addDataPointBatch(data_matrix) 

4500

In [9]:
# Create an index
start = time.time()
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 30, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}
Indexing time = 9.202667


In [10]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params) 

Setting query-time parameters {'efSearch': 100}


In [11]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=1.021121 (sec), per query=0.002042 (sec), per query adjusted for thread number=0.008169 (sec)


In [12]:
nbrs[0]

(array([   3, 2302, 2324, 2829,  836, 2521, 4276, 1075, 3219, 1804, 3329,
         555, 2714, 4470, 2264, 1774, 4087, 3945, 3730, 1070,  665, 3762,
        2347,  392, 2336, 1347, 3745,  108, 2922, 3663, 3424, 3320, 1292,
        1982, 3984, 1246, 3537, 2215, 3356, 3569,   12, 4433,  449,  401,
        3926,  132, 1153,   31,  941,  125, 1310,  769,  450,  743, 2170,
        3838, 4285, 2429,  518,  114, 3416,  263, 1716, 2200, 2353, 3120,
         255, 2253,  111, 1717, 2193, 4212, 4389, 1413, 4052,  709, 4024,
        4219, 1370, 1293, 4439,  452, 2544, 2311, 3068, 2636,  978, 3355,
        1810, 1747,  172,  152, 1417,  748, 2949, 3630, 2599, 3511, 4035,
        3211], dtype=int32),
 array([0.6513385 , 0.7716905 , 0.80642194, 0.8168692 , 0.84857154,
        0.8595936 , 0.8656192 , 0.87085605, 0.8720833 , 0.87907964,
        0.8833531 , 0.8887931 , 0.90145725, 0.9019042 , 0.91259426,
        0.91409534, 0.914767  , 0.91586804, 0.91947174, 0.92219555,
        0.92302996, 0.9237418 , 0

In [13]:
# Computing gold-standard data 
print('Computing gold-standard data')

start = time.time()
sindx = NearestNeighbors(n_neighbors=K, metric='cosine', algorithm='brute').fit(data_matrix)
end = time.time()

print('Brute-force preparation time %f' % (end - start))

start = time.time() 
gs = sindx.kneighbors(query_matrix)
end = time.time()

print('brute-force kNN time total=%f (sec), per query=%f (sec)' % 
      (end-start, float(end-start)/query_qty) )

Computing gold-standard data
Brute-force preparation time 0.020844
brute-force kNN time total=0.643384 (sec), per query=0.001287 (sec)


In [14]:
# Finally computing recall
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.970600


In [15]:
# Save a meta index and the data
index.saveIndex('sparse_index.bin', save_data=True)

In [16]:
# Re-intitialize the library, specify the space, the type of the vector
newIndex = nmslib.init(method='hnsw', space='cosinesimil_sparse', data_type=nmslib.DataType.SPARSE_VECTOR) 

In [17]:
# Re-load the index and the data
newIndex.loadIndex('sparse_index.bin', load_data=True)

In [18]:
# Setting query-time parameters and querying
print('Setting query-time parameters', query_time_params)
newIndex.setQueryTimeParams(query_time_params)

query_qty = query_matrix.shape[0]
start = time.time() 
new_nbrs = newIndex.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

Setting query-time parameters {'efSearch': 100}
kNN time total=0.938577 (sec), per query=0.001877 (sec), per query adjusted for thread number=0.007509 (sec)


In [19]:
# Finally computing recall for the new result set
recall=0.0
for i in range(0, query_qty):
  correct_set = set(gs[1][i])
  ret_set = set(new_nbrs[i][0])
  recall = recall + float(len(correct_set.intersection(ret_set))) / len(correct_set)
recall = recall / query_qty
print('kNN recall %f' % recall)

kNN recall 0.970600
